In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [6]:
file = "train"
df_set = pd.read_csv(f"D:\\diplom\\{file}_ship_segmentations_v2.csv")
df_set['HasShip'] = df_set['EncodedPixels'].notnull()

sdf = df_set.loc[lambda df_set : df_set['HasShip'] == True].reset_index(drop=True)

In [3]:
rle = lambda x: ' '.join(x.split(" ")[1::2]) + ' ' + \
' '.join([ str(int(hor) % 256)  for hor in x.split(" ")[0::2]]) if (isinstance(x, str)) else x

clean_sdf = sdf.copy()
clean_sdf["RleAndPosition"] = clean_sdf.EncodedPixels.apply(rle)

In [ ]:
same_rle = lambda x: clean_sdf.ImageId[clean_sdf["RleAndPosition"] == x].tolist()
clean_sdf["AllSameRle"] = clean_sdf["RleAndPosition"].apply(same_rle)

In [ ]:
un_clean_sdf = clean_sdf.groupby('ImageId')['AllSameRle'].apply(lambda x: set(x.sum())) 

In [ ]:
Dropped = []
un_clean_sdf_copy =  un_clean_sdf.copy()
for item in un_clean_sdf.iteritems():
    if not item[0] in Dropped:
        for item_check in un_clean_sdf.iteritems():
            if item_check[0] in item[1] and not item_check[0] in Dropped and item_check[0] != item[0]:
                un_clean_sdf = un_clean_sdf.drop(item_check[0])
                Dropped = Dropped + [item_check[0]]
print(len(un_clean_sdf))

In [ ]:
train_un_clean_sdf = un_clean_sdf.sample(frac = 0.8, random_state = 768)
validation_un_clean_sdf = un_clean_sdf.drop(train_un_clean_sdf.index)

In [ ]:
all_uniq_ship_segmentations = clean_sdf.loc[[True if ID in un_clean_sdf.index \
                                            else False for ID in clean_sdf["ImageId"]]]
all_uniq_ship_segmentaions.to_csv(r"D:\\diplom\\un_ship_segmentaions.csv", index = None)

In [ ]:
un_train_ship_segmentations = clean_sdf.loc[[True if ID in train_un_clean_sdf.index \
                                             else False for ID in clean_sdf["ImageId"]]]
un_train_ship_segmentations.to_csv(r"D:\\diplom\\un_train_ship_segmentaions.csv", index = None)

In [ ]:
un_validation_ship_segmentations = clean_sdf.loc[[True if ID in validation_un_clean_sdf.index \
                                                 else False for ID in clean_sdf["ImageId"]]]
un_validation_ship_segmentations.to_csv(r"D:\\diplom\\un_validation_ship_segmentaions.csv", index = None)